In [1]:
%matplotlib inline
import sys
sys.path.append("..")
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import fiona
import pyproj
import shapely.geometry
import shapely.ops
from rtree import index

from cafo import utils

In [2]:
src_crs = pyproj.CRS("epsg:4326")
dst_crs = pyproj.CRS('epsg:26918')
project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform

## Load test set polygons

In [3]:
test_shapes = []
with fiona.open("../data/delmarva_testing_set_polygons.geojson") as f:
    print(f.crs)
    for row in f:
        test_shapes.append(shapely.geometry.shape(
            row["geometry"]
        ))

{'init': 'epsg:26918'}


## Helper methods

In [4]:
def load_predictions(fn):
    predicted_shapes_all = []
    predicted_shapes_filtered = []
    with fiona.open(fn) as f:
        for row in f:
            shape = shapely.geometry.shape(row["geometry"])
            shape = shapely.ops.transform(project,shape)
            predicted_shapes_all.append(shape)
            if utils.filter_polygon(**row["properties"]):
                predicted_shapes_filtered.append(shape)
    return predicted_shapes_all, predicted_shapes_filtered

In [5]:
def do_experiment(test_shapes, predicted_shapes):
    
    test_shapes_index = index.Index()
    for i, shape in enumerate(test_shapes):
        test_shapes_index.insert(i, shape.bounds)
        
    predicted_shape_index = index.Index()
    for i, shape in enumerate(predicted_shapes):
        predicted_shape_index.insert(i, shape.bounds)
        
    test_shape_ious = []
    for i, test_shape in enumerate(test_shapes):
        max_iou = 0
        for j in predicted_shape_index.intersection(test_shape.bounds):
            predicted_shape = predicted_shapes[j] 
            iou = test_shape.intersection(predicted_shape).area / test_shape.union(predicted_shape).area
            max_iou = max(iou, max_iou)
        test_shape_ious.append(max_iou)
        
    predicted_shape_ious = []
    for i, predicted_shape in enumerate(predicted_shapes):
        max_iou = 0
        for j in test_shapes_index.intersection(predicted_shape.bounds):
            test_shape = test_shapes[j] 
            iou = predicted_shape.intersection(test_shape).area / predicted_shape.union(test_shape).area
            max_iou = max(iou, max_iou)
        predicted_shape_ious.append(max_iou)

    return np.array(test_shape_ious), np.array(predicted_shape_ious)

In [6]:
def get_precision_recall(test_shape_ious, predicted_shape_ious, alpha=0.5):
    
    tp = (predicted_shape_ious>alpha).sum()
    
    fn = (test_shape_ious<=alpha).sum()
    fp = predicted_shape_ious.shape[0] - tp
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    return precision, recall

## Run experiments

In [8]:
%%time
rows = []
for fn in glob.glob("../output/inference/*.geojson"):
    print(fn)
    predicted_shapes_all, predicted_shapes_filtered = load_predictions(fn)

    test_shape_ious, predicted_shape_ious = do_experiment(test_shapes, predicted_shapes_filtered)
    precision_filtered, recall_filtered = get_precision_recall(test_shape_ious, predicted_shape_ious, 0.5)
    
    test_shape_ious, predicted_shape_ious = do_experiment(test_shapes, predicted_shapes_all)
    precision, recall = get_precision_recall(test_shape_ious, predicted_shape_ious, 0.5)
        
        
    results = {
        "precision": precision,
        "recall": recall,
        "precision_filtered": precision_filtered,
        "recall_filtered": recall_filtered
    }
    parameters = utils.parse_fn_parts(os.path.basename(fn).replace(".geojson",""))
    results.update(parameters)
    rows.append(results)

../output/inference/train-single_unet_0.1_0.01_rotation.geojson
../output/inference/train-all_unet_0.05_0.01.geojson
../output/inference/train-augment_unet_0.5_0.01_rotation.geojson
../output/inference/train-all_unet_0.5_0.01_rotation.geojson
../output/inference/train-augment_unet_0.1_0.01_rotation.geojson
../output/inference/train-single_unet_0.1_0.01.geojson
../output/inference/train-single_unet_0.5_0.01_rotation.geojson
../output/inference/train-all_unet_0.5_0.01.geojson
../output/inference/train-all_unet_0.05_0.01_rotation.geojson
../output/inference/train-augment_unet_0.05_0.01_rotation.geojson
../output/inference/train-single_unet_0.5_0.01.geojson
../output/inference/train-augment_unet_0.5_0.01.geojson
../output/inference/train-augment_unet_0.05_0.01.geojson
../output/inference/train-single_unet_0.05_0.01_rotation.geojson
../output/inference/train-all_unet_0.1_0.01_rotation.geojson
../output/inference/train-all_unet_0.1_0.01.geojson
../output/inference/train-augment_unet_0.1_0.01

In [10]:
df = pd.DataFrame(rows)

In [11]:
df

,precision,recall,precision_filtered,recall_filtered,training_set,model,negative_sample_probability,lr,rotation
0,0.159547,0.968836,0.805982,0.934396,train-single,unet,0.10,0.01,True
1,0.374250,0.972502,0.894831,0.938318,train-all,unet,0.05,0.01,False
2,0.488972,0.960369,0.920074,0.931326,train-augment,unet,0.50,0.01,True
3,0.360616,0.966055,0.875671,0.924457,train-all,unet,0.50,0.01,True
4,0.377412,0.967033,0.885057,0.936389,train-augment,unet,0.10,0.01,True
5,0.366347,0.980734,0.897007,0.951447,train-single,unet,0.10,0.01,False
6,0.431402,0.877140,0.866242,0.842975,train-single,unet,0.50,0.01,True
7,0.650432,0.967003,0.942326,0.940576,train-all,unet,0.50,0.01,False
8,0.325560,0.963201,0.855679,0.937325,train-all,unet,0.05,0.01,True
9,0.273500,0.972527,0.870497,0.946828,train-augment,unet,0.05,0.01,True


In [14]:
(df["precision_filtered"] - df["precision"]).mean()

0.48009896534258967

In [15]:
(df["recall_filtered"] - df["recall"]).mean()

-0.03258631501795023

In [12]:
df.to_csv("../results/filtering_results_iou50.csv", index=False)